In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Importacion de librerías
import pandas as pd
import numpy as np
# Gestionar las rutas
import utils.paths as path

In [3]:
raw_data_dir = path.make_dir_function(["data","raw"])
ratings_dir = raw_data_dir("ratings")
csv_file = ratings_dir.glob("*.csv")

In [4]:
# Directorios de los datasets
csv_dict = {}
for csv_file in csv_file:
    csv_dict[f"{csv_file.name.split('.')[0]}"] = csv_file

In [5]:
# Lectura de datos en dataframe
dict_data = {}
for file_name, file_dir in zip(csv_dict.keys(), csv_dict.values()):
    dict_data[file_name] = pd.read_csv(file_dir,index_col=False)

In [6]:
# Concatenando datasets
def concat_data_frames(dict_datasets):
    df = pd.concat(dict_datasets.values(), keys=dict_datasets.keys())
    df = df.reset_index(drop=True)
    return df

In [7]:
# Concatenando los dataframes de rating
ratings_df = concat_data_frames(dict_data)

In [10]:
# Funcion para renombrar la "rating" por "score"
def rename_rating_col(df: pd.DataFrame):
    df = df.rename(columns={"rating":"score"})
    return df

In [11]:
ratings_df = rename_rating_col(ratings_df)

In [12]:
ratings_df.head()

,userId,score,timestamp,movieId
0,1,1.0,1425941529,as680
1,1,4.5,1425942435,ns2186
2,1,5.0,1425941523,hs2381
3,1,5.0,1425941546,ns3663
4,1,5.0,1425941556,as9500


In [18]:
import datetime

def normalize_timestamp_col(df):
    # Convertir la columna timestamp en segundos "s" a un objeto datetime
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit='s')
    # Formatear la columna como "YYYY-MM-DD"
    df["timestamp"] = df["timestamp"].dt.strftime('%Y-%m-%d')
    return df

In [19]:
# Cambiando el formtato defecha a "YYYY-MM-DD"
ratings_df = normalize_timestamp_col(ratings_df)

In [20]:
# Veamos como quedó el formato de los datos de fecha
ratings_df.head(2)

,userId,score,timestamp,movieId
0,1,1.0,2015-03-09,as680
1,1,4.5,2015-03-09,ns2186


Antes de exportar los datos verifiquemos la cantidad de campos nulos por columna y los registro duplicados.

In [24]:
# Verificación de valores nulos
ratings_df.isna().sum()

userId       0
score        0
timestamp    0
movieId      0
dtype: int64

In [21]:
# Verificacion de cantidad de registros duplicados
ratings_df.duplicated().sum()

10466

In [27]:
# Remover los registros duplicados
ratings_df = ratings_df.drop_duplicates().reset_index(drop=True)

In [28]:
# Cantidad de registros duplicados despues de remover
ratings_df.duplicated().sum()

0

In [40]:
# Cambiando de tipo de datos numérico float64 a float32
ratings_df['score'] = ratings_df['score'].astype(np.float32)

In [42]:
# Cambiando de tipo de datos numérico int64 a int32
ratings_df['userId'] = ratings_df['userId'].astype(np.int32)

In [43]:
# Exportación de la data
def save_as_parquet(df,file_name):
    data_proc_dir = path.make_dir_function(["data","interim"])
    file_path = data_proc_dir(file_name)
    df.to_parquet(f"{file_path}.parquet")

In [49]:
# Data guardado en /data/interim'
save_as_parquet(ratings_df,"ratings_v1")

In [45]:
# Informacion del dataset exportado
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11013823 entries, 0 to 11013822
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int32  
 1   score      float32
 2   timestamp  object 
 3   movieId    object 
dtypes: float32(1), int32(1), object(2)
memory usage: 252.1+ MB


In [47]:
# Estadísticos básicos de las variables numéricas
ratings_df.describe()

,userId,score
count,1.101382e+07,1.101382e+07
mean,8.997060e+04,3.533449e+00
std,8.686389e+04,1.059798e+00
min,1.000000e+00,5.000000e-01
25%,2.855700e+04,3.000000e+00
50%,5.684500e+04,3.500000e+00
75%,1.168640e+05,4.000000e+00
max,2.708960e+05,5.000000e+00


_**Nota**: Función para normalizar la columna ``movieIdG`` con datasets iniciales_

In [ ]:
# def normalize_movied_id(df):
#     # {'net': 'ns', 'ama': 'as', 'dis': 'ds', 'hul': 'hs'}
#     initials_list = df["movieIdG"].str[:3].unique()
#     initials_dict = {}
    
#     for elem in initials_list:
#         initials_dict[elem] = f"{elem[0]}s"
    
#     for old_initial, new_inital in initials_dict.items():
#         # Garantiza que la función replace solo se aplique a valores que contengan el string antiguo, evitando así errores.
#         df["movieIdG"] = df["movieIdG"].apply(lambda x: x.replace(old_initial,new_inital) if old_initial in x else x)
#     # Renombrar la columna "movieIdG" por "movieId"
#     df = df.rename(columns={"movieIdG":"movieId"})
#     return df